We will need:

beautifulsoup4
pandas

Furthermore, we will need all the links gathered in one place for convinience

In [190]:
import bs4
import pandas
import urllib
links = {"Oral19":r"https://2019.ic2s2.org/oral-presentations/","Poster19":r"https://2019.ic2s2.org/posters/","OralAndPoster20":r"https://docs.google.com/spreadsheets/d/e/2PACX-1vTX9_1Xftn7D-nSI8X9b7tafr_Z0kAbphKdfZ8qUSU9p-syXNsGPdhHl5ZyTnKKL-T6dCEJqtsrn3wy/pubhtml?gid=181378784&single=true&widget=true&headers=false","OralAndPoster21":r"https://easychair.org/smart-program/IC2S2-2021/talk_author_index.html"}

def getWebpagesBS4(link):
    page = urllib.request.urlopen(link)
    return bs4.BeautifulSoup(page)

First we need to get a list of all the different presentations, then we can treat each presentation individually

In [191]:
oral2019BS = getWebpagesBS4(links["Oral19"])
listOfAllPresentations = oral2019BS.find("div","main").findChild("div").findChild("div").findChild("article").findChild("div").findChild("div","col-md-8").findAll("p")[3:]

We can see, that the pattern for each table in the oral presentation 2019 is as follows:

Scenario a)
<em>
Chair: NAME
</em>

Scenario b)
<br>
TIME0 - TIME1 - NAME0,NAME1,...,NAMEN. TITLE
</br>
or
TIME0 - TIME1 - No Presentation


In [192]:
#Retruns a string where TIME0 - TIME1 - has been removed
def getTextAfterTimestamp(text):
    incrementer = 0
    idx_end_of_timeStamp = 0
    for idx,c in enumerate(text):
        if (c == "–"):
            incrementer += 1
            if (incrementer == 2):
                idx_end_of_timeStamp = idx + 1

    return text[idx_end_of_timeStamp::]

#Returns a set of names from a string formatted as: NAME0,NAME1,...,NAMEn{.,-}
def getNamesFromText(text):
    text = text[1:]
    endingChars = {".":True,"–":True}
    result_set = set()
    current_name_string = ""

    for idx,c in enumerate(text):
        if(endingChars.get(c,False) and not text[idx-1].isupper()): #We check to see if the char is an ending char, and the letter before it wasn't an initial
            result_set.add(current_name_string)
            return result_set
        elif (c == ","):
            result_set.add(current_name_string)
            current_name_string = ""
        else:
            if(c.isalpha() or len(current_name_string)>0): #We don't want spacebar at the start of the names
                current_name_string += c

    return result_set

def getNameFromChair(text):
    text = text.split(":")
    name = text[1]
    nameFiltered = ""

    for chr in name:
        if(chr.isalpha() or len(nameFiltered)>0):
            nameFiltered+= chr

    return nameFiltered

We start off by getting all the names of the chairs, as we can use BS4 to get these quite easily:

In [193]:
set_of_names = set()
for presentations in listOfAllPresentations:
    chairAndName = presentations.find("em").string
    set_of_names.add(getNameFromChair(chairAndName))

We then move unto getting all the presenters

In [194]:
for presentations in listOfAllPresentations:
    text = str(presentations)
    presentations_list = text.split("<")
    for presentation in presentations_list:
        if (len(presentation)>0 and presentation[0] == "b"): #We now have the start of each break
            presentation_text = getTextAfterTimestamp(presentation)
            if(not "No presentation" in presentation_text):
                set_of_names = set_of_names.union(getNamesFromText(presentation_text))

print(set_of_names)

{'Abe Hofman', 'Nadia Metoui', 'David Darmon ', 'Geoffrey Canright', 'Chaoming Song', 'Rudy Arthur', 'Peter Van Aelst', 'Yifang Ma', 'Cristian Candia Vallejos', 'Timo Hartmann', 'Simon A. Levin', 'Hywel Williams', 'David Rozado', 'Manuel Cebrian', 'Ramona Roller', 'Ethan Bernstein', 'Kinga Makovi', 'Alex Pentland ', 'Fırat Duruşan', 'David Garcia ', 'János Kertész', 'Angelo Cardoso', 'Scott Hale', 'Theresa Gessler', 'Angel Sánchez', 'Abhishek Samantray', 'Johannes Gruber', 'Wojtek Przepiorka', 'Esteban Moro', 'Sai Hitesh Jonnalagadda', 'Tian Yang', 'Wei Lee Woon', 'Arnout van de Rijt', 'Jeremy Burke', 'Mel Stanfill', 'Mitsuhiro Odaka', 'Marco De Nadai', 'Takanori Nishida', 'Ashton Anderson', 'Joseph Bajjalieh', 'Kai Larsen', 'Ulrich Matter', 'Fernando Diaz', 'Przemyslaw Grabowicz', 'Laura Bringmann', 'Rachel Winter', 'Milena Tsvetkova ', 'Marco Brambilla', 'Joana Mendonça', 'James Evans', 'Andrea Ballatore', 'Alexandra Pang', 'Sune Lehmann Jørgensen', 'Philip Leifeld', 'Daniele Guariso

We can see a couple of errors, where we've scraped the title of the presentation as well as the presenter, we will remove these manually


In [195]:
set_of_names.remove("r/>The Higher Education Space")
set_of_names.remove("Moved to 3D Text Analysis) Ivan Smirnov")
set_of_names.add("Ivan Smirnov")

In [196]:
bs2021 = getWebpagesBS4(links["OralAndPoster21"])

In [197]:
listOfEntries = bs2021.find("div",{"id":"content"}).findAll("tr",{"class":"entry"})


In [198]:
def findEndingIndex(first_and_last_names_String):
    state = "q0"

    for idx,char in enumerate(first_and_last_name_String):
        if(char == "<" and state == "q0"):
            state = "<"
        elif(char == "/" and state == "<"):
            state = "/"
        elif(char == "a" and state == "/"):
            state = "a"
        elif (char == ">" and state == "a"):
            return idx - 3
        else:
            state = "q0"

    return "error"

def findStartingIndex(first_and_last_names_String):
    state = "q0"

    for idx,char in enumerate(first_and_last_name_String):
        if(char == "<" and state == "q0"):
            state = "<"
        elif(char == "/" and state == "<"):
            state = "/"
        elif(char == "s" and state == "/"):
            state = "s"
        elif(char == "p" and state == "s"):
            state = "p"
        elif(char == "a" and state == "p"):
            state = "a"
        elif(char == "n" and state == "a"):
            state = "n"
        elif (char == ">" and state == "n"):
            state = ">"
        elif(char == "," and state == ">"):
            return idx + 1
        else:
            state = "q0"

    return "error"

def getFirstName(first_and_last_String):
    starting_idx = findStartingIndex(first_and_last_name_String)
    ending_idx = findEndingIndex(first_and_last_name_String)
    return first_and_last_String[starting_idx:ending_idx]



In [199]:
for entry in listOfEntries:
    first_and_last_name = entry.find("td",{"class":"name"})
    first_and_last_name_String = str(first_and_last_name)
    first_name = getFirstName(first_and_last_name_String)
    last_name = first_and_last_name.find("span").string
    if(first_name != "error" and last_name != "error"):
        if(first_name[0] == " "):
            first_name = first_name[1:]

        if(last_name[0] == " "):
            full_name = first_name + last_name
        else:
            full_name = first_name + " " + last_name

        set_of_names.add(full_name)

We then convert our set to a dataframe, sort it to check for duplicates and then we save it.

In [200]:
import pandas as pd
import csv

In [205]:
list_of_names = list(set_of_names)
df = pd.DataFrame(list_of_names)
df.sort_values(0)
df.columns = ["Names"]
df.to_csv(r"C:\Users\DripTooHard\PycharmProjects\SocialInformatics\Week1\listOfCompSocNamesWeek1",index=False)